# Segmenting and Clustering Neighborhoods in Toronto

## Install necessary libraries

In [6]:
!pip install beautifulsoup4
!pip install lxml
!pip install requests
from bs4 import BeautifulSoup
import requests
import pandas as pd

## Webscraping

#### Bring in Beautifulsoup package

In [239]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')

#### Parse html and remove "Not assigned"

In [240]:
Nhoods_df = pd.DataFrame(columns = ['PostalCode', 'Borough', 'Neighborhood'])

#table = soup.find('table', class_= 'wikitable sortable jquery-tablesorter')
table = soup.find('table')
#table_body = table.find('tbody') #tbody is the name on wikipedia
table_body = table.tbody

rows = table_body.find_all('tr')

for row in rows:
    cols = row.find_all('td')
    counter = -1
    # Placeholder for each row entry
    table_split = pd.DataFrame([["Empty", "Empty", "Empty"]], columns = ['PostalCode', 'Borough', 'Neighborhood'])
    
    #Start assuming row will not be rejected
    reject = 0
    
    # Loop through all columns in each row
    for col in cols:
        counter = counter + 1
        # Last column had /n at the back of everything
        if counter == 2:
            new_text = col.get_text()[:-1]
            # Check if neighbourhood missing
            if new_text == "Not assigned":
                new_text = table_split.iloc[0,counter - 1]
        else:
            new_text = col.get_text()
        table_split.iloc[0,counter] = new_text
            
        # Remove "Not assigned", start with blank slate of reject = 0
        if new_text == "Not assigned" and counter == 1:
            reject = 1 
    
    if reject == 0:
        Nhoods_df = Nhoods_df.append(table_split)
        
Nhoods_df = Nhoods_df[1:]
    
Nhoods_df.to_csv(r'C:\Users\Tim Blake\Documents\Work\Training\IBM data science specialization\Final Capstone Project\test2.csv')


#### Combining neighborhoods

In [241]:
Nhoods_df_merge = Nhoods_df.groupby(by=['PostalCode','Borough']).agg(lambda x: ','.join(x))
Nhoods_df_merge.reset_index(level=['PostalCode','Borough'], inplace=True)
Nhoods_df_merge


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie..."
101,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam..."


#### Give shape of dataframe

In [236]:
Nhoods_df.shape

(209, 3)